# Question 3

In [2]:
import os.path
import random
import time
from functools import partial
from tkinter import *
import sys
import heapq

from lib.EightPuzzle import EightPuzzle

# h1 = manhattan distance function

In [3]:
def manhattan_distance(state):
    goal_state = [1, 2, 3, 4, 5, 6, 7, 8, 0]

    goal_coordinates = {
        1: (0, 0), 2: (0, 1), 3: (0, 2),
        4: (1, 0), 5: (1, 1), 6: (1, 2),
        7: (2, 0), 8: (2, 1), 0: (2, 2) 
    }
    
    distance = 0
    for i in range(3):
        for j in range(3):
            tile = state[3*i + j] 
            if tile != 0:
                goal_i, goal_j = goal_coordinates[tile]
                distance += abs(i - goal_i) + abs(j - goal_j)

    return distance


# h2 = no of misplaced tiles

In [4]:
def misplaced_tiles(state):
    goal_state = [1, 2, 3, 4, 5, 6, 7, 8, 0]

    misplaced_count = sum(1 for i, j in zip(state, goal_state) if i != j)

    return misplaced_count


In [5]:
root = Tk()

state = [1, 2, 3, 4, 5, 6, 7, 8, 0]
puzzle = EightPuzzle(tuple(state))
solution = None

b = [None] * 9

# Greedy Best-First search with h1 = Manhattan distance

In [6]:
def solve_h1():
    global puzzle
    p = puzzle.initial
    
    nodes_explored = 0
    max_frontier_size = 1

    frontier = [(manhattan_distance(p), 0, [p])] 
    explored = set()
    
    while frontier:
        _, cost, path = heapq.heappop(frontier)
        node = path[-1] 
        
        if puzzle.goal_test(node):
            print("Time Complexity (Nodes Explored):", nodes_explored)
            print("Space Complexity (Max Frontier Size):", max_frontier_size)
            print("Cost of the Path:", cost)
            return path
        explored.add(node)
        nodes_explored += 1
        
        max_frontier_size = max(max_frontier_size, len(frontier))
        
        for action in puzzle.actions(node):
            child = puzzle.result(node, action)
            if child not in explored:
                priority = manhattan_distance(child)
                heapq.heappush(frontier, (priority, cost + 1, path + [child]))  # Add child state to the path with updated cost
    
    print("No path found")
    return []


# Greedy best first search with H2= no of misplaced tiles

In [7]:
def solve_h2():
    global puzzle
    p = puzzle.initial
    nodes_explored = 0
    max_frontier_size = 1

    frontier = [(misplaced_tiles(p), 0, [p])] 
    explored = set()
    
    while frontier:
        _, cost, path = heapq.heappop(frontier)
        node = path[-1]  # Get the current state from the end of the path
        
        if puzzle.goal_test(node):
            print("Time Complexity (Nodes Explored):", nodes_explored)
            print("Space Complexity (Max Frontier Size):", max_frontier_size)
            print("Cost of the Path:", cost) 
            return path
        explored.add(node)
        nodes_explored += 1
        max_frontier_size = max(max_frontier_size, len(frontier))
        
        for action in puzzle.actions(node):
            child = puzzle.result(node, action)
            if child not in explored:
                priority = misplaced_tiles(child)
                heapq.heappush(frontier, (priority, cost + 1, path + [child]))
    
    print("No path found")
    return []


# A* search: use the cost function given in Q1(iii, for UCS)

In [8]:
def cost_estimate(state):
    goal_state = [1, 2, 3, 4, 5, 6, 7, 8, 0]  
    cost = 0
    for current, target in zip(state, goal_state):
        if current != target:
            diff = abs(state.index(current) - goal_state.index(current))
            if diff == 1: 
                cost += 1
            elif diff == 3: 
                cost += 2
            elif diff == 2:
                cost += 3
            elif diff == 4:
                cost += 4
    return cost

def solve_astar():
    global puzzle
    p = puzzle.initial
    
    nodes_explored = 0
    max_frontier_size = 1
    
    frontier = [(cost_estimate(p), 0, [p])]
    explored = set()
    
    while frontier:
        _, cost, path = heapq.heappop(frontier)
        node = path[-1]
        
        if puzzle.goal_test(node):
            print("Time Complexity (Nodes Explored):", nodes_explored)
            print("Space Complexity (Max Frontier Size):", max_frontier_size)
            print("Cost of the Path:", cost)
            return path
        explored.add(node)
        nodes_explored += 1
    
        max_frontier_size = max(max_frontier_size, len(frontier))
        
        for action in puzzle.actions(node):
            child = puzzle.result(node, action)
            if child not in explored:
                priority = len(path) + cost_estimate(child)
                heapq.heappush(frontier, (priority, cost + 1, path + [child]))
    
    print("No path found")
    return []


# A* search using h1 = Manhattan distance 

In [9]:
def solve_astar_h1():
    global puzzle
    p = puzzle.initial
    
    nodes_explored = 0
    max_frontier_size = 1
    
    frontier = [(manhattan_distance(p), 0, [p])] 
    explored = set()
    
    while frontier:
        _, cost, path = heapq.heappop(frontier)
        node = path[-1]  # Get the current state from the end of the path
        
        if puzzle.goal_test(node):
            print("Time Complexity (Nodes Explored):", nodes_explored)
            print("Space Complexity (Max Frontier Size):", max_frontier_size)
            print("Cost of the Path:", cost)
            return path
        explored.add(node)
        nodes_explored += 1
        
        max_frontier_size = max(max_frontier_size, len(frontier))
        
        for action in puzzle.actions(node):
            child = puzzle.result(node, action)
            if child not in explored:
                priority = len(path) + manhattan_distance(child) 
                heapq.heappush(frontier, (priority, cost + 1, path + [child])) 
                
    print("No path found")
    return []


# A* with h2 = no. of misplaced tiles

In [10]:
def solve_astar_h2():
    global puzzle
    p = puzzle.initial
    
    nodes_explored = 0
    max_frontier_size = 1

    frontier = [(misplaced_tiles(p), 0, [p])]
    explored = set()
    
    while frontier:
        _, cost, path = heapq.heappop(frontier)
        node = path[-1] 
        
        if puzzle.goal_test(node):
            print("Time Complexity (Nodes Explored):", nodes_explored)
            print("Space Complexity (Max Frontier Size):", max_frontier_size)
            print("Cost of the Path:", cost) 
            return path
        explored.add(node)
        nodes_explored += 1
        max_frontier_size = max(max_frontier_size, len(frontier))
        
        for action in puzzle.actions(node):
            child = puzzle.result(node, action)
            if child not in explored:
                priority = len(path) + misplaced_tiles(child) 
                heapq.heappush(frontier, (priority, cost + 1, path + [child])) 
    
    print("No path found")
    return []


In [13]:
def scramble():
    """Scrambles the puzzle starting from the goal state"""

    global state
    global puzzle
    possible_actions = ['UP', 'DOWN', 'LEFT', 'RIGHT']
    scramble = []
    for _ in range(60):
        scramble.append(random.choice(possible_actions))

    for move in scramble:
        if move in puzzle.actions(state):
            state = list(puzzle.result(state, move))
            puzzle = EightPuzzle(tuple(state))
            create_buttons()

def solve_steps():
    """Solves the puzzle step by step using Greedy Best-First search with Manhattan distance heuristic"""

    global puzzle
    global state
    print("Select the search algorithm:")
    print("1. Greedy Search with Manhatan distance as hearistic function")
    print("2. Greedy Search with  No of misplaced tiles as hearistic function")
    print("3. A* search with the  cost function")
    print("4. A* with Manhatan distance as hearistic function")
    print("5. A* with  No of misplaced tiles as hearistic function")
    search_option = int(input("Enter the number of the desired search algorithm: "))

    if search_option == 1:
        solution = solve_h1()
        print("Path length : ", len(solution))
    elif search_option == 2:
        solution = solve_h2()
        print("Path length : ", len(solution))
    elif search_option == 3:
        solution = solve_astar()
        print("Path length : ", len(solution))
    elif search_option == 4:
        solution = solve_astar_h1()
        print("Path length : ", len(solution))
    elif search_option == 5:
        solution = solve_astar_h2()
        print("Path length : ", len(solution))
    else:
        print("Enter valid input")
        solution = []

    
    if solution:
        for move in solution[1:]: 
            state = move
            create_buttons()
            root.update()
            time.sleep(0.75)
    else:
        print("No solution found.")

def exchange(index):
    """Interchanges the position of the selected tile with the zero tile under certain conditions"""

    global state
    global solution
    global puzzle
    zero_ix = list(state).index(0)
    actions = puzzle.actions(state)
    current_action = ''
    i_diff = index // 3 - zero_ix // 3
    j_diff = index % 3 - zero_ix % 3
    if i_diff == 1:
        current_action += 'DOWN'
    elif i_diff == -1:
        current_action += 'UP'

    if j_diff == 1:
        current_action += 'RIGHT'
    elif j_diff == -1:
        current_action += 'LEFT'

    if abs(i_diff) + abs(j_diff) != 1:
        current_action = ''

    if current_action in actions:
        b[zero_ix].grid_forget()
        b[zero_ix] = Button(root, text=f'{state[index]}', width=6, font=('Helvetica', 40, 'bold'),
                            command=partial(exchange, zero_ix))
        b[zero_ix].grid(row=zero_ix // 3, column=zero_ix % 3, ipady=40)
        b[index].grid_forget()
        b[index] = Button(root, text=None, width=6, font=('Helvetica', 40, 'bold'), command=partial(exchange, index))
        b[index].grid(row=index // 3, column=index % 3, ipady=40)
        state[zero_ix], state[index] = state[index], state[zero_ix]
        puzzle = EightPuzzle(tuple(state))


def create_buttons():
    """Creates dynamic buttons"""

    # TODO: Find a way to use grid_forget() with a for loop for initialization
    b[0] = Button(root, text=f'{state[0]}' if state[0] != 0 else None, width=6, font=('Helvetica', 40, 'bold'),
                  command=partial(exchange, 0))
    b[0].grid(row=0, column=0, ipady=40)
    b[1] = Button(root, text=f'{state[1]}' if state[1] != 0 else None, width=6, font=('Helvetica', 40, 'bold'),
                  command=partial(exchange, 1))
    b[1].grid(row=0, column=1, ipady=40)
    b[2] = Button(root, text=f'{state[2]}' if state[2] != 0 else None, width=6, font=('Helvetica', 40, 'bold'),
                  command=partial(exchange, 2))
    b[2].grid(row=0, column=2, ipady=40)
    b[3] = Button(root, text=f'{state[3]}' if state[3] != 0 else None, width=6, font=('Helvetica', 40, 'bold'),
                  command=partial(exchange, 3))
    b[3].grid(row=1, column=0, ipady=40)
    b[4] = Button(root, text=f'{state[4]}' if state[4] != 0 else None, width=6, font=('Helvetica', 40, 'bold'),
                  command=partial(exchange, 4))
    b[4].grid(row=1, column=1, ipady=40)
    b[5] = Button(root, text=f'{state[5]}' if state[5] != 0 else None, width=6, font=('Helvetica', 40, 'bold'),
                  command=partial(exchange, 5))
    b[5].grid(row=1, column=2, ipady=40)
    b[6] = Button(root, text=f'{state[6]}' if state[6] != 0 else None, width=6, font=('Helvetica', 40, 'bold'),
                  command=partial(exchange, 6))
    b[6].grid(row=2, column=0, ipady=40)
    b[7] = Button(root, text=f'{state[7]}' if state[7] != 0 else None, width=6, font=('Helvetica', 40, 'bold'),
                  command=partial(exchange, 7))
    b[7].grid(row=2, column=1, ipady=40)
    b[8] = Button(root, text=f'{state[8]}' if state[8] != 0 else None, width=6, font=('Helvetica', 40, 'bold'),
                  command=partial(exchange, 8))
    b[8].grid(row=2, column=2, ipady=40)


def create_static_buttons():
    """Creates scramble and solve buttons"""

    scramble_btn = Button(root, text='Scramble', font=('Helvetica', 30, 'bold'), width=8, command=partial(init))
    scramble_btn.grid(row=3, column=0, ipady=10)
    solve_btn = Button(root, text='Solve', font=('Helvetica', 30, 'bold'), width=8, command=partial(solve_steps))
    solve_btn.grid(row=3, column=2, ipady=10)

1
def init():
    """Calls necessary functions"""
    global state
    global solution
    state = [1, 2, 3, 4, 5, 6, 7, 8, 0]
    scramble()
    create_buttons()
    create_static_buttons()

init()
root.mainloop()


start
Select the search algorithm:
1. Greedy Search with Manhatan distance as hearistic function
2. Greedy Search with  No of misplaced tiles as hearistic function
3. A* search with the  cost function
4. A* with Manhatan distance as hearistic function
5. A* with  No of misplaced tiles as hearistic function


Enter the number of the desired search algorithm:  1


Time Complexity (Nodes Explored): 6
Space Complexity (Max Frontier Size): 5
Cost of the Path: 6
Path length :  7
